In [1]:
from camembert import Bri
import pysam
import random
import os

# Comparison of BRI and IndexedReads

In [2]:
bam_big = './wgEncodeUwRepliSeqSknshS3AlnRep1.bam'  # 904M
bam_small = './wgEncodeUwRepliSeqK562G1AlnRep1.bam'  # 40M
bam_long = './rel3-nanopore-wgs-4177064552-FAB42260.fastq.gz.sorted.bam' # 2GB long reads
test_file = bam_long

## Benchmark BRI index build time
*This inludes also disk I/O for saving .bri file

In [3]:
%%timeit
bri = Bri(test_file)
bri.create()

# release memory immediately
del bri

13.2 s ± 182 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
bam_size = os.stat(test_file).st_size >> 20
bri_size = os.stat(test_file + '.bri').st_size >> 20
print("Bam size={}MB, read index size={}MB".format(bam_size, bri_size))

Bam size=2021MB, read index size=22MB


## Benchmark IndexedReads build time

In [5]:
%%timeit
hts = pysam.AlignmentFile(test_file)
indexed = pysam.IndexedReads(hts)
indexed.build()

# release memory immediately
del indexed
del hts

12.7 s ± 64.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Get random read names from our test file

In [6]:
hts = pysam.AlignmentFile(test_file)
names = [r.query_name for r in hts]
test_reads = random.choices(names, k=1000)

In [7]:
indexed = pysam.IndexedReads(hts)
indexed.build()

## Benchmark IndexedReads seek time

In [8]:
%%timeit
for r in test_reads:
    list(indexed.find(r))

643 ms ± 7.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
bri = Bri(test_file)
bri.load()

## Benchmark BRI seek time

In [10]:
%%timeit
for r in test_reads:
    list(bri.get(r))

645 ms ± 8.12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
